In [ ]:
# Importación de librerías necesarias
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import SPSAOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math

# Definición de parámetros del circuito cuántico
num_qubits = 4
num_layers = 2

# Configuración del dispositivo cuántico
dev = qml.device("default.qubit", wires=num_qubits)

# Etapa del Feature Map: Preparación del estado cuántico
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

# Etapa del Ansatz: Uso de BasicEntanglerLayers
# BasicEntanglerLayers aplica rotaciones de un parámetro en cada qubit seguidas de una cadena cerrada de puertas CNOT.
def ansatz(weights):
    qml.BasicEntanglerLayers(weights, wires=range(num_qubits))

# Definición del circuito cuántico
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)
    ansatz(weights)
    return qml.expval(qml.PauliZ(0))

# Función del clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Etapa de Función de Pérdida: Definición de la función de pérdida
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

# Etapa de evaluación de la Función de Costo: Definición de la función de costo
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# Etapa de Análisis de Precisión: Definición de la función de precisión
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

# Preprocesamiento de los datos de entrada
df_train = pd.read_csv('train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

# División de datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived']
)

# Conversión de datos a formato compatible con PennyLane
X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# Pesos iniciales definidos
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

# Configuración del optimizador SPSA
opt = SPSAOptimizer(maxiter=70)
num_it = 70
batch_size = math.floor(len(X_train) / num_it)

# Entrenamiento del clasificador cuántico
weights = weights_init
bias = bias_init
for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias = opt.step(lambda w, b: cost(w, b, X_batch, Y_batch), weights, bias)

    # Evaluación de precisión durante el entrenamiento
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )

# Pesos finales entrenados
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

# Evaluación de desempeño del clasificador cuántico final
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
print("Accuracy:", accuracy_score(Y_test, predictions))
print("Precision:", precision_score(Y_test, predictions))
print("Recall:", recall_score(Y_test, predictions))
print("F1 Score:", f1_score(Y_test, predictions, average='macro'))


Iter:     1 | Cost: 2.3967006 | Accuracy: 0.3657928 
Iter:     2 | Cost: 2.2644488 | Accuracy: 0.3657928 
Iter:     3 | Cost: 2.2534620 | Accuracy: 0.3657928 
Iter:     4 | Cost: 2.0162662 | Accuracy: 0.3657928 
Iter:     5 | Cost: 1.9804642 | Accuracy: 0.3657928 
Iter:     6 | Cost: 1.9779986 | Accuracy: 0.3657928 
Iter:     7 | Cost: 1.8367779 | Accuracy: 0.3657928 
Iter:     8 | Cost: 1.8588699 | Accuracy: 0.3657928 
Iter:     9 | Cost: 1.8501784 | Accuracy: 0.3657928 
Iter:    10 | Cost: 1.7616114 | Accuracy: 0.3657928 
Iter:    11 | Cost: 1.6894425 | Accuracy: 0.3657928 
Iter:    12 | Cost: 1.5969791 | Accuracy: 0.3657928 
Iter:    13 | Cost: 1.5764041 | Accuracy: 0.3657928 
